# PV Notebook
This notebook will explore solar generation around the ERCOT 4CP events.

In [32]:
import pandas as pd
import psycopg2
import sqlalchemy as sqla

In [33]:
# get our DB connection
engine = sqla.create_engine('postgresql://{}:{}@{}:{}/{}'.format('steve',
                                                                     'steve123abc',
                                                                     '10.20.10.50',
                                                                     5433,
                                                                     'dataport'
                                                                     ))


In [34]:
# dates acquired from http://mis.ercot.com/misapp/GetReports.do?reportTypeId=13037&reportTitle=Planned%20Service%20Four%20Coincident%20Peak%20Calculations&showHTMLView=&mimicKey

event_start_dates = ['2019-06-19 17:00:00-05', '2019-07-30 16:30:00-05', '2019-08-12 17:00:00-05', '2019-09-06 16:45:00-05',
               '2018-06-27 17:00:00-05', '2018-07-19 17:00:00-05', '2018-08-23 16:45:00-05', '2018-09-19 16:30:00-05',
               '2017-06-23 16:45:00-05', '2017-07-28 17:00:00-05', '2017-08-16 17:00:00-05', '2017-09-20 16:45:00-05',
               '2016-06-15 17:00:00-05', '2016-07-14 16:00:00-05', '2016-08-11 16:30:00-05', '2016-09-19 16:16:00-05'
              ]
event_end_dates = ['2019-06-19 17:15:00-05', '2019-07-30 16:45:00-05', '2019-08-12 17:15:00-05', '2019-09-06 17:00:00-05',
               '2018-06-27 17:15:00-05', '2018-07-19 17:15:00-05', '2018-08-23 17:00:00-05', '2018-09-19 16:45:00-05',
               '2017-06-23 17:00:00-05', '2017-07-28 17:15:00-05', '2017-08-16 17:15:00-05', '2017-09-20 17:00:00-05',
               '2016-06-15 17:15:00-05', '2016-07-14 16:15:00-05', '2016-08-11 16:45:00-05', '2016-09-19 16:31:00-05']

In [35]:
# dataids, pv direction, amount of PV of solar homes
query = """
select dataid, pv, pv_panel_direction, total_amount_of_pv, amount_of_west_facing_pv, amount_of_south_facing_pv
from other_datasets.metadata
where pv is not null
and total_amount_of_pv is not null
and grid is not null 
and solar is not null
and pv_panel_direction in ('South', 'West')
and egauge_1min_min_time < '2016-09-19'
and egauge_1min_max_time > '2019-06-19'
"""

# create a dataframe with the data from the sql query
df = pd.read_sql_query(sqla.text(query), engine)
df

,dataid,pv,pv_panel_direction,total_amount_of_pv,amount_of_west_facing_pv,amount_of_south_facing_pv
0,2337,yes,South,4.995,0.000,4.995
1,7030,yes,South,5.250,0.000,5.250
2,9982,yes,South,3.675,NaN,3.675
3,7767,yes,South,5.920,0.000,5.920
4,890,yes,South,6.250,0.000,6.250
5,4357,yes,South,4.000,0.000,4.000
6,7541,yes,South,6.250,0.000,6.250
7,7719,yes,South,6.300,NaN,6.300
8,6691,yes,South,5.200,NaN,5.200
9,3649,yes,South,7.540,NaN,7.540


In [36]:
# grab dataids and convert them to a string to put into the SQL query
dataids_list = df['dataid'].tolist()
dataids_list
dataids_str = ','.join(list(map(str, dataids_list)))
dataids_str

'2337,7030,9982,7767,890,4357,7541,7719,6691,3649,3506,5357,8419,8086,8467,7800,7741,7739,5796,5738,5129,2470,1169,187,9932,6139,93,5439,171,5892,6643,2199,9134,8767,8626,8317,8084,7940,6836,5972,4336,3482,3039,3009,2814,2461,2233,744,661'

In [37]:
# calculate usage grid minus solar (which is actually grid + solar because solar is negative use)
# 
query_2 = """
select dataid, localminute, solar, grid from electricity.eg_realpower_1min 
where localminute >= '2019-06-19 17:00:00-05' and localminute <= '2019-06-19 17:15:00-05'
and dataid in """

query_2 = query_2 + "({})".format(dataids_str)
print(query_2)

# create a dataframe with the data from the sql query
df2 = pd.read_sql_query(sqla.text(query_2), engine)

# Calculate the difference and add is as a new column
df2['solar+grid'] = df2.apply(lambda row: row.solar + row.grid, axis=1)
df2.head(15)


select dataid, localminute, solar, grid from electricity.eg_realpower_1min 
where localminute >= '2019-06-19 17:00:00-05' and localminute <= '2019-06-19 17:15:00-05'
and dataid in (2337,7030,9982,7767,890,4357,7541,7719,6691,3649,3506,5357,8419,8086,8467,7800,7741,7739,5796,5738,5129,2470,1169,187,9932,6139,93,5439,171,5892,6643,2199,9134,8767,8626,8317,8084,7940,6836,5972,4336,3482,3039,3009,2814,2461,2233,744,661)


,dataid,localminute,solar,grid,solar+grid
0,93,2019-06-19 22:00:00+00:00,1.518,-1.221,0.297
1,93,2019-06-19 22:01:00+00:00,1.519,-1.221,0.298
2,93,2019-06-19 22:02:00+00:00,1.509,-1.212,0.297
3,93,2019-06-19 22:03:00+00:00,1.496,-1.200,0.296
4,93,2019-06-19 22:04:00+00:00,1.484,-1.188,0.296
5,93,2019-06-19 22:05:00+00:00,1.476,-1.076,0.400
6,93,2019-06-19 22:06:00+00:00,1.466,-1.097,0.369
7,93,2019-06-19 22:07:00+00:00,1.452,2.668,4.120
8,93,2019-06-19 22:08:00+00:00,1.438,3.166,4.604
9,93,2019-06-19 22:09:00+00:00,1.415,3.266,4.681


In [38]:
# group by dataid and average(?) the values
grouped = df2.groupby(['dataid']).mean()
grouped

,solar,grid,solar+grid
dataid,,,
93,1.422375,1.340375,2.762750
171,3.845813,-0.614375,3.231438
187,3.432437,1.802187,5.234625
661,2.013375,-1.533750,0.479625
744,1.523438,2.341500,3.864937
890,-0.002000,4.737938,4.735938
1169,1.352250,-1.017188,0.335062
2199,1.177625,-0.890500,0.287125
2233,0.882188,3.590000,4.472187


In [39]:
# let's bring in the metadata on the dataids
joined = pd.merge(grouped, df, on='dataid')
joined

,dataid,solar,grid,solar+grid,pv,pv_panel_direction,total_amount_of_pv,amount_of_west_facing_pv,amount_of_south_facing_pv
0,93,1.422375,1.340375,2.762750,yes,South,6.105,0.000,6.105
1,171,3.845813,-0.614375,3.231438,yes,West,6.250,6.250,NaN
2,187,3.432437,1.802187,5.234625,yes,West,5.250,5.250,NaN
3,661,2.013375,-1.533750,0.479625,yes,South,6.300,NaN,6.300
4,744,1.523438,2.341500,3.864937,yes,South,5.920,0.000,5.920
5,890,-0.002000,4.737938,4.735938,yes,South,6.250,0.000,6.250
6,1169,1.352250,-1.017188,0.335062,yes,South,3.885,0.000,3.885
7,2199,1.177625,-0.890500,0.287125,yes,South,4.995,0.000,4.995
8,2233,0.882188,3.590000,4.472187,yes,South,6.250,0.000,6.250
9,2337,1.526000,-1.212750,0.313250,yes,South,4.995,0.000,4.995


In [40]:
west_vs_south = joined.groupby(['pv_panel_direction']).mean()
west_vs_south

,dataid,solar,grid,solar+grid,total_amount_of_pv,amount_of_west_facing_pv,amount_of_south_facing_pv
pv_panel_direction,,,,,,,
South,5525.485714,1.646084,0.668120,2.314204,5.378714,0.000000,5.378714
West,4417.833333,3.539844,-0.185682,3.354161,6.307917,6.307917,0.000000
